<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Optim" data-toc-modified-id="Optim-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Optim</a></span><ul class="toc-item"><li><span><a href="#Optimizerの種類" data-toc-modified-id="Optimizerの種類-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Optimizerの種類</a></span></li><li><span><a href="#どれを使うのか?" data-toc-modified-id="どれを使うのか?-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>どれを使うのか?</a></span></li><li><span><a href="#Learning-rateの調整" data-toc-modified-id="Learning-rateの調整-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Learning rateの調整</a></span></li></ul></li><li><span><a href="#重みの初期値" data-toc-modified-id="重みの初期値-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>重みの初期値</a></span></li><li><span><a href="#Batch-Normalization" data-toc-modified-id="Batch-Normalization-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Batch Normalization</a></span></li><li><span><a href="#畳み込みニューラルネットワーク" data-toc-modified-id="畳み込みニューラルネットワーク-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>畳み込みニューラルネットワーク</a></span><ul class="toc-item"><li><span><a href="#畳み込み層" data-toc-modified-id="畳み込み層-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>畳み込み層</a></span></li></ul></li><li><span><a href="#重要なネットワーク構造" data-toc-modified-id="重要なネットワーク構造-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>重要なネットワーク構造</a></span><ul class="toc-item"><li><span><a href="#VGG" data-toc-modified-id="VGG-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>VGG</a></span></li><li><span><a href="#GoogLeNet" data-toc-modified-id="GoogLeNet-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>GoogLeNet</a></span></li><li><span><a href="#ResNet" data-toc-modified-id="ResNet-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>ResNet</a></span></li><li><span><a href="#PyTorchで利用可能なモデル" data-toc-modified-id="PyTorchで利用可能なモデル-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>PyTorchで利用可能なモデル</a></span></li></ul></li><li><span><a href="#ユーティリティ関係" data-toc-modified-id="ユーティリティ関係-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>ユーティリティ関係</a></span><ul class="toc-item"><li><span><a href="#結果の保存" data-toc-modified-id="結果の保存-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>結果の保存</a></span></li></ul></li></ul></div>

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 14

In [2]:
import torch
import torchvision

## Optim

**使い方:** 

- `optimizer`をインスタンス化し、勾配情報からparametersを更新する。
- すべてのoptimizerは`step`を実装しており、これを実行すればパラメータが更新される
    - `optimizer.step()`を実行
    - `backward()`を実行して勾配を計算したあと一度だけ実行する。
    - Conjugate GradientやLBFGSなどの最適化アルゴリズムを使用する場合は異なる。(割愛)



### Optimizerの種類

- `Adadelta`
- `Adagrad`
    - パラメータ一つ一つに対して個別のlearning rateを適応させる
    - "適応": AdaptiveからAdagrad
    - 値が大きく更新されたパラメータの学習率は小さくしていく
    - 具体的に言うと、これまでの勾配の2乗の値が蓄積されていき、勾配をその平方根で除して更新する
    - パラメータ更新の度に勾配の2乗の値は蓄積されて無限大になるので、やがて更新値は0となる
        - これを改善したのがRMSprop
- `Adam`
    - Adaptive momentum estimation
    - 要は、Adagrad + RMSprop(Momenumも含む?)
    - 詳細は、*Kingma*らの原著論文を参照して(arXiv:1412.6980)
- `AdamW`
- `SparseAdam`
- `Adamax`
- `ASGD`
- `LBFGS`
- `RMSprop`
    - "Adagrad"での勾配情報の二乗値の蓄積部分を指数移動平均に変更したもの
    - 直近の更新情報がより重み付けされることになる
    - Adagradと違ってすべてを蓄積するわけではないので、更新値が0となることはない
- `Rprop`
- `SGD`
    - 確率的勾配降下法
    - データからミニバッチを無作為に取り出しそのデータから勾配降下法を用いてパラメータ更新
    - なので"確率的"という名前がつく
    - データ全体に対して行う勾配降下法は"バッチ勾配降下法"と呼称されることも
    - 勾配の方向が極小点を指していないので、パラメータ更新が非常に非効率
    - その結果として、損失関数は更新の過程で大きく振動するようすがみられる
    - Pytorchでは本来のSGDというよりは、学習率が固定されたOptimizerという意味?
    - つまり、データの入力は1セットではなく、ミニバッチが前提となっている

### どれを使うのか?

- 決定的な方法は無いし、選定の指標も無い(と思う)
- 単純にSGDが使われる場合も多い
- ただ、一般的にはSGDよりも他の手法の方が収束は速いし精度が出る傾向にある
- Adamは結構やるやつ
- それ以外でもはまるケースはある

---
**結論: まずはAdam試しとけ。精度が出ない、もしくは時間があるなら他の手法も試してみろ。**

### Learning rateの調整

`torch.optim.lr_scheduler`を使用してエポックごとのlearning rateの調整が可能

## 重みの初期値

**重みの初期値の選定は非常に重要な問題**

不適切な初期値を利用すると、勾配消失が起こり、学習がまったく進まなくなってしまう。
(損失関数が全然減らなくなってしまう)

- Xavierの初期値
    - 活性化関数がシグモイド関数の場合に有効な手法
- Heの初期値
    - 活性化関数がReLU関数の場合に有効な手法

## Batch Normalization

活性化後の値に偏りがあると、学習がうまくいかない。(勾配消失、など)

強制的に値に分布をもたせてやれば、学習が早く進むのでは?というアイデアが、**Batch Normalization**

※Courseraのコースでは、Week3でこの内容の講義があった。

---
[利点]

- 学習が速い
- 初期値依存があまりない
    - Batch Normによって調整されるので、初期値の分布を気にする必要性が小さくなる
- 化学臭が抑制される
    - その他の過学習抑制方法を使用しなくてもよくなる
    
---

- **通常の正規化(Normalization)**: 入力データに対して平均を0、分散を1に調整する
- **Batch Normalization**: すべての隠れ層への入力に対してNormalizationを行う。また、その後にバイアスの加算とスケーリングが行われる。
    - 通常は、活性化前にNormalizationを行うことが一般的
    - 各層の活性はミニバッチごとに変わるので、ミニバッチごとにNormalizationを行う。
    - 重みがある層(FC層など)と活性化層(ReLUなど)の間に新しくBatch Norm層が入るイメージ
    - 当然、backpropのときにもこれを考慮して計算する必要がある

---
[Batch Normの計算]

$$
Z_{\mathrm{norm}}^{(i)} = \frac{Z^{(i)} - \mu}{\sqrt{\sigma^2 + \epsilon}}
$$

これは、平均が0で分散が1のパラメータになる。

**ただし、隠れ層の出力では、この分布(平均0、分散1)が常に適しているとは限らない。**
なので、以下のように、任意の分布となるようにパラメータを変換してもいい。

$$
\tilde{Z}^{(i)} = \gamma Z_{\mathrm{norm}}^{(i)} + \beta
$$

$\tilde{Z}^{(i)}$ は平均$\beta$、分散$\gamma$の分布をもつことになる。

これを活性化関数にぶちこみ、この層からの出力が得られる。

- ミニバッチサイズ
    - ハイパーパラメータ

## 畳み込みニューラルネットワーク

主に以下の層から構成される

- Convolution layer
- Pooling layer
- Full connected(FC) layer

### 畳み込み層

- **出力される画像(2次元データ)のサイズ**: カーネルサイズやpadding, strideの大きさによって一意に決まる
- **出力されるデータのチャンネル数**: 畳み込み層のチャンネル数。これは任意の値に指定する。

一般的には、2次元方向のサイズはどんどん小さくしていき、チャンネル数を増やしていく、というネットワーク構造が多い。
(つまり、2次元の大きな入力データを、畳み込み層をかましながら1次元のデータにしていく、という流れ)
その後、Affine層をかまして出力を得る。

とある畳み込み層について

- kernelサイズ: $f$
    - kernelのチャンネル数は、必ずこの層への入力データのチャンネル数に等しくする
- paddingの数: $p$
- strideの数: $s$

これらはネットワーク構築の際に設定する。

※ kernelの中身は学習の対象。

この畳み込み層に対して、$n \times h \times w$のデータが入力され、
$n^{'} \times h{'} \times w{'}$
のデータが出力される、とする。

このうち、$h^{'} \times w^{'}$は$f, p, s$と$h, w$によって決まる。

$$
h^{'} \times w^{'} = 
\left( \lfloor \frac{h + 2p - f}{s} \rfloor + 1 \right) 
\times
\left( \lfloor \frac{w + 2p - f}{s} \rfloor + 1 \right)
$$

これを計算するためには入力データのチャンネル分のカーネルが必要となる。
(この場合は、$n \times f \times f$のカーネルを、
入力データ$n \times h \times w$に対して適用し、
$1 \times h^{'} \times w^{'}$が得られる)

このように、1つのカーネルからは1チャンネルの二次元データが得られることになる。
出力チャンネル数は、このカーネルの個数によって決まる。

$n^{'}$については、ネットワークを構築する際に決める。
(一般的に、ネットワークが深くなるにつれて、$n^{'}$の値を大きくする。)


## 重要なネットワーク構造

- VGG
- GoogLeNet
- ResNet

### VGG

- PoolingやStride、kernelサイズ、フィルター数は固定しておく(ユーザーは調整しない)
- シンプルな構造なので学習しやすく、応用がききやすい

### GoogLeNet

- Inception構造を基本としたネットワーク構造

### ResNet

### PyTorchで利用可能なモデル

- `AlexNet`
- `VGG`
- `ResNet`
- `SqueezeNet`
- `DenseNet`
- `Inception v3`
- `GoogLeNet`
- `ShuffleNet v2`
- `MobileNet v2`
- `ResNeXt`
- `Wide ResNet`
- `MNASNet`

## ユーティリティ関係

### 結果の保存

- ベストプラクティスは、`torch.save`を使用して、ネットワークの`state_dict()`を保存する。
- ロードする場合は、ネットワークをインスタンス化して、`load_state()`を実行

詳細は、
[ここ](https://pytorch.org/docs/stable/notes/serialization.html)
に記載がある。